In [1]:
import gc
import numpy as np
import pandas as pd
matrix = pd.read_csv('./matrix.csv')
#print(matrix)

In [2]:
for num in range(8):
    temp_1 = matrix['session_'+ str(num) + '_merchant_id']
    temp_2 = matrix['session_'+ str(num) + '_action_type']
    temp_3 = 'session_'+ str(num) + '_merchant_id'
    temp_4 = 'session_'+ str(num) + '_action_type'
    for num_1 in range(len(temp_1)):
        temp = temp_1[num_1]
        #print(temp)
        merchant_id_list = temp[1:-1].strip().split(', ')
        #print(merchant_id_list)
        matrix.at[num_1,temp_3] = merchant_id_list

    for num_2 in range(len(temp_2)):
        temp = temp_2[num_2]
        action_type_list = temp[1:-1].strip().split(', ')
        matrix.at[num_2, temp_4] = action_type_list

#print(matrix['session_0_merchant_id'][0])

In [3]:
M=500

#print(matrix['session_0_merchant_id'][0])
#print(type(matrix['session_0_merchant_id'][0]))
#print(matrix['session_0_merchant_id'])

for num in range(8):
    temp_1 = matrix['session_'+ str(num) + '_merchant_id']
    temp_2 = matrix['session_'+ str(num) + '_action_type']
    temp_3 = 'session_'+ str(num) + '_merchant_id'
    temp_4 = 'session_'+ str(num) + '_action_type'
    for merchant_id_num1 in range(len(temp_1)):
        temp_5 = temp_1[merchant_id_num1]
        #print(temp_5)
        if len(temp_5)>M:
            matrix.at[merchant_id_num1,temp_3] = temp_5[:M]
        else:
            while len(temp_5)<M:
                temp_5.append('0')
            matrix.at[merchant_id_num1,temp_3] = temp_5
    for action_type_num1 in range(len(temp_1)):
        temp_6 = temp_2[action_type_num1]
        if len(temp_6)>M:
            matrix.at[action_type_num1,temp_4] = temp_6[:M]
        else:
            while len(temp_6)<M:
                temp_6.append('0')
            matrix.at[action_type_num1,temp_4] = temp_6            


In [4]:
matrix = matrix.drop('merchant_id_y',axis = 1)
matrix = matrix.drop('action_type',axis = 1)
matrix = matrix.drop('time_stamp',axis = 1)
matrix = matrix.rename(columns={'session_0_merchant_id':'sess_0_merchant_id'})
matrix = matrix.rename(columns={'session_0_action_type':'sess_0_action_type'})
matrix = matrix.rename(columns={'session_1_merchant_id':'sess_1_merchant_id'})
matrix = matrix.rename(columns={'session_1_action_type':'sess_1_action_type'})
matrix = matrix.rename(columns={'session_2_merchant_id':'sess_2_merchant_id'})
matrix = matrix.rename(columns={'session_2_action_type':'sess_2_action_type'})
matrix = matrix.rename(columns={'session_3_merchant_id':'sess_3_merchant_id'})
matrix = matrix.rename(columns={'session_3_action_type':'sess_3_action_type'})
matrix = matrix.rename(columns={'session_4_merchant_id':'sess_4_merchant_id'})
matrix = matrix.rename(columns={'session_4_action_type':'sess_4_action_type'})
matrix = matrix.rename(columns={'session_5_merchant_id':'sess_5_merchant_id'})
matrix = matrix.rename(columns={'session_5_action_type':'sess_5_action_type'})
matrix = matrix.rename(columns={'session_6_merchant_id':'sess_6_merchant_id'})
matrix = matrix.rename(columns={'session_6_action_type':'sess_6_action_type'})
matrix = matrix.rename(columns={'session_7_merchant_id':'sess_7_merchant_id'})
matrix = matrix.rename(columns={'session_7_action_type':'sess_7_action_type'})
matrix = matrix.rename(columns={'merchant_id_x':'merchant_id'})

In [5]:
matrix = matrix.drop('sess_3_merchant_id',axis = 1)
matrix = matrix.drop('sess_3_action_type',axis = 1)
matrix = matrix.drop('sess_4_merchant_id',axis = 1)
matrix = matrix.drop('sess_4_action_type',axis = 1)
matrix = matrix.drop('sess_5_merchant_id',axis = 1)
matrix = matrix.drop('sess_5_action_type',axis = 1)
matrix = matrix.drop('sess_6_merchant_id',axis = 1)
matrix = matrix.drop('sess_6_action_type',axis = 1)
matrix = matrix.drop('sess_7_merchant_id',axis = 1)
matrix = matrix.drop('sess_7_action_type',axis = 1)

In [6]:
for num in range(len(matrix)):
    for num_1 in range(3):
        temp_1 = matrix['sess_'+ str(num_1) + '_merchant_id']
        temp_2 = temp_1[num]
        temp_3 = temp_2[0]
        if temp_3 == '0':
            matrix.loc[num,'sess_length'] = num_1
            break
        else:
            if num_1 == 2 :
                matrix.loc[num,'sess_length'] = 3

#print(matrix)

In [7]:
# 分割训练数据和测试数据
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']

In [8]:
# 使用DSIN模型
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import log_loss
from deepctr.inputs import SparseFeat,VarLenSparseFeat,DenseFeat,get_feature_names
from deepctr.models import DIN, DIEN, DSIN
from sklearn.metrics import classification_report

train_X['action_type']=3
feature_columns = []
for column in train_X.columns:
    session_text = 'sess'
    if session_text not in column :
        #print(column)
        num = train_X[column].nunique()
        if num > 10000:
            dim = 10
        else:
            if num > 1000:
                dim = 8
            else:
                dim = 4
        #print(num)
        if column  == 'user_id':
            feature_columns += [SparseFeat(column, 19111+1, embedding_dim=dim, use_hash=True)]
        elif column  == 'merchant_id':
            feature_columns += [SparseFeat(column, 4994+1, embedding_dim=dim, use_hash=True)]
        elif column  == 'action_type':
            feature_columns += [SparseFeat(column, 4+1, embedding_dim=dim, use_hash=True)]
        else:
            feature_columns += [DenseFeat(column, 1)]

print('M=', M)

C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\tensorflow\python\framework\dty

M= 500


In [9]:
# maxlen为历史信息的长度，vocabulary_size为onehot的长度
feature_columns += [VarLenSparseFeat(SparseFeat('sess_0_merchant_id', vocabulary_size=19111 + 1, embedding_dim=8, use_hash=True, embedding_name='merchant_id'),maxlen=M),
                    VarLenSparseFeat(SparseFeat('sess_0_action_type', vocabulary_size=4 + 1, embedding_dim=4, use_hash=True, embedding_name='cate_id'),maxlen=M)]
feature_columns += [VarLenSparseFeat(SparseFeat('sess_1_merchant_id', vocabulary_size=19111 + 1, embedding_dim=8, use_hash=True, embedding_name='merchant_id'),maxlen=M),
                    VarLenSparseFeat(SparseFeat('sess_1_action_type', vocabulary_size=4 + 1, embedding_dim=4, use_hash=True, embedding_name='cate_id'),maxlen=M)]
feature_columns += [VarLenSparseFeat(SparseFeat('sess_2_merchant_id', vocabulary_size=19111 + 1, embedding_dim=8, use_hash=True, embedding_name='merchant_id'),maxlen=M),
                    VarLenSparseFeat(SparseFeat('sess_2_action_type', vocabulary_size=4 + 1, embedding_dim=4, use_hash=True, embedding_name='cate_id'),maxlen=M)]
#feature_columns += [VarLenSparseFeat(SparseFeat('sess_3_merchant_id', vocabulary_size=19111 + 1, embedding_dim=8, use_hash=True, embedding_name='merchant_id'),maxlen=M),
                    #VarLenSparseFeat(SparseFeat('sess_3_action_type', vocabulary_size=4 + 1, embedding_dim=4, use_hash=True, embedding_name='cate_id'),maxlen=M)]
#feature_columns += [VarLenSparseFeat(SparseFeat('sess_4_merchant_id', vocabulary_size=19111 + 1, embedding_dim=8, use_hash=True, embedding_name='merchant_id'),maxlen=M),
                    #VarLenSparseFeat(SparseFeat('sess_4_action_type', vocabulary_size=4 + 1, embedding_dim=4, use_hash=True, embedding_name='cate_id'),maxlen=M)]
#feature_columns += [VarLenSparseFeat(SparseFeat('sess_5_merchant_id', vocabulary_size=19111 + 1, embedding_dim=8, use_hash=True, embedding_name='merchant_id'),maxlen=M),
                    #VarLenSparseFeat(SparseFeat('sess_5_action_type', vocabulary_size=4 + 1, embedding_dim=4, use_hash=True, embedding_name='cate_id'),maxlen=M)]
#feature_columns += [VarLenSparseFeat(SparseFeat('sess_6_merchant_id', vocabulary_size=19111 + 1, embedding_dim=8, use_hash=True, embedding_name='merchant_id'),maxlen=M),
                    #VarLenSparseFeat(SparseFeat('sess_6_action_type', vocabulary_size=4 + 1, embedding_dim=4, use_hash=True, embedding_name='cate_id'),maxlen=M)]
#feature_columns += [VarLenSparseFeat(SparseFeat('sess_7_merchant_id', vocabulary_size=19111 + 1, embedding_dim=8, use_hash=True, embedding_name='merchant_id'),maxlen=M),
                    #VarLenSparseFeat(SparseFeat('sess_7_action_type', vocabulary_size=4 + 1, embedding_dim=4, use_hash=True, embedding_name='cate_id'),maxlen=M)]


In [11]:
hist_features=['merchant_id','action_type']
# 使用DSIN模型
model=DSIN(feature_columns, hist_features, sess_max_count=3, att_embedding_size=3, att_head_num=4)
# 使用Adam优化器，二分类的交叉熵
model.compile('adam', 'binary_crossentropy', metrics=['binary_crossentropy'])

# 组装train_model_input，得到feature names，将train_X转换为字典格式
feature_names=list(train_X.columns)
train_model_input = {name:train_X[name].values for name in feature_names}
# histroy输入必须是二维数组
from tqdm import tqdm
for fea in ['sess_0_merchant_id','sess_0_action_type',
           'sess_1_merchant_id','sess_1_action_type',
           'sess_2_merchant_id','sess_2_action_type']:
           #'sess_3_merchant_id','sess_3_action_type',
           #'sess_4_merchant_id','sess_4_action_type',
           #'sess_5_merchant_id','sess_5_action_type',
           #'sess_6_merchant_id','sess_6_action_type',
           #'sess_7_merchant_id','sess_7_action_type']:
    l = []
    for i in tqdm(train_model_input[fea]):
        l.append(i)
    train_model_input[fea]=np.array(l)
history = model.fit(train_model_input, train_y, verbose=True, epochs=3, validation_split=0.2,batch_size=32)

100%|███████████████████████████████████████████████████████████████████████| 17837/17837 [00:00<00:00, 1691510.11it/s]


Train on 14269 samples, validate on 3568 samples
Epoch 1/3
14269/14269 [==============================] - 1938s 136ms/sample - loss: 0.2270 - binary_crossentropy: 0.2270 - val_loss: 0.2299 - val_binary_crossentropy: 0.2299
Epoch 2/3
14269/14269 [==============================] - 1908s 134ms/sample - loss: 0.2246 - binary_crossentropy: 0.2246 - val_loss: 0.2283 - val_binary_crossentropy: 0.2283
Epoch 3/3
14269/14269 [==============================] - 1911s 134ms/sample - loss: 0.2245 - binary_crossentropy: 0.2245 - val_loss: 0.2293 - val_binary_crossentropy: 0.2293


In [13]:
# 转换test__model_input
test_data['action_type']=3
test_model_input = {name:test_data[name].values for name in feature_names}
from tqdm import tqdm
for fea in ['sess_0_merchant_id','sess_0_action_type',
           'sess_1_merchant_id','sess_1_action_type',
           'sess_2_merchant_id','sess_2_action_type']:
    l = []
    for i in tqdm(test_model_input[fea]):
        l.append(i)
    test_model_input[fea]=np.array(l)


# 得到预测结果
prob = model.predict(test_model_input)
submission = pd.read_csv('./data_format1_small/test.csv')
submission['prob'] = prob
submission.to_csv('prediction.csv', index=False)


100%|█████████████████████████████████████████████████████████████████████████| 6056/6056 [00:00<00:00, 1926777.29it/s]
